In [10]:
import os

f = open('train_file_name.txt','a')
path = '/ncudeep/yuan/Defect_0214/0928-0214/2800_DBC_0928-0214'
for root,dirs,files in os.walk(path):
    for file in files:
        f.write(file+'\n')

f.close()

In [1]:
# merge and rename
import shutil
import os
path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/ori"
despath = "/msl/home/kelly08385/Pictures/Fast_defect/4800/ori"

for root,dirs,files in os.walk(path):
    for file in files:
        try:
            #print(root.split('/')[8])
            shutil.move(os.path.join(root,file),os.path.join(os.path.join(despath,root.split('/')[8],file[:-4]+'.jpg')))
        except:
            print("continue")

In [6]:
import xml.etree.cElementTree as ET
import os,shutil

image_path = "/msl/home/kelly08385/Pictures/Fast_defect/2800/2800_no_type"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/2800/Annotations"

for root,dirs,files in os.walk(image_path):
    for file in files:
        #dir = root.split('/')[-1]
        # exist xml
        if os.path.exists(os.path.join(xml_path,file[:-4]+".xml")):
            tree = ET.parse(os.path.join(xml_path,file[:-4]+'.xml'))
            tree_root = tree.getroot()
            for e in tree_root.iter("name"):
                change_dir = e.text 
                #print(change_dir)
                    
            _despath = os.path.join(image_path,change_dir)
            #print(_despath)
            mkdir(_despath)
            shutil.move(os.path.join(root,file),os.path.join(_despath,file))
        

In [2]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
## use original image and crop image to find xml 

import xml.etree.cElementTree as ET
import cv2
import os
import numpy as np
import shutil

crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/crop"
des_crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/train_crop"
ori_path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/ori"
des_path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/data"
des_path2 = "/msl/home/kelly08385/Pictures/Fast_defect/4800/JPEGImages"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/4800/Annotations"

#quiz_path = "/home/kelly/Pictures/defect/0312_work/9300/quiz"

In [ ]:
for root,dirs,files in os.walk(crop_path):
    dir = root.split("/")[-1]
    for file in files:
        mkdir(os.path.join(des_path,dir))
        mkdir(os.path.join(des_crop_path,dir))
        if os.path.exists(os.path.join(ori_path,dir,file[:-4]+".jpg")):

            try:
                cropImg = cv2.imread(os.path.join(root,file))
                cropImg = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
                oriImg = cv2.imread(os.path.join(ori_path,dir,file[:-4]+".jpg"))
                oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2GRAY)
                
                result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
                x_and_y = np.unravel_index(result.argmax(),result.shape)
                print(x_and_y)

                tree = ET.parse('/msl/home/kelly08385/Pictures/trained_defect/total/Annotations/2300AAHL5AHYC14C100007-03.xml')
                tree_root = tree.getroot()
                print(tree_root)
                change_dict = {"filename":file,"width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                               "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split("/")[-1]}
                for key in change_dict.keys():
                    for e in tree_root.iter(key):
                        e.text = str(change_dict[key])
                shutil.copy(os.path.join(ori_path,dir,file),os.path.join(des_path2,file))
                shutil.move(os.path.join(ori_path,dir,file),os.path.join(des_path,dir,file))
                shutil.move(os.path.join(root,file),os.path.join(des_crop_path,dir,file))
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
            except:
                print("contunue")

In [ ]:
# find red rec and move to the data and delete xml
# PPP0 PC07 PF01 PF02
import os,shutil
base_path = "/msl/home/kelly08385/Pictures/Fast_defect/4300/quiz"
des_path = "/msl/home/kelly08385/Pictures/Fast_defect/4300/rec"
# xml_path = "/home/kelly/Pictures/fastrcnn/2300_0322/result/Annotations"

for root,dirs,files in os.walk(base_path):
    dir = root.split('/')[-1]
    print(dir)
    if dir == "S10G":
        for file in files:
            cropImg = getImgInRedRect(os.path.join(root,file))
            if not cropImg is None:
                print(os.path.join(root,file))
                _despath = os.path.join(des_path,dir)
                mkdir(_despath)
    #             print(os.path.join(xml_path,file[:-4]+".xml"))
                shutil.move(os.path.join(root,file),os.path.join(_despath,file))
    #             os.remove(os.path.join(xml_path,file[:-4]+".xml"))
